In [ ]:
import numpy as np
import scipy as sp
import pandas as pd

from scipy import optimize

%matplotlib inline 
import matplotlib.pyplot as plt

from simul_functions import generate_toy_data, get_bootstrap_data, set_intensity, get_data_to_fit
from simul_functions import get_attenuation_parameters


In [ ]:
# input parameters for the simualtions
minE = 10**15 #eV
maxE = 10**18 #eV
gamma = -2.5 # slope of the spectrum

#relation between energy and s38
A = 10**12
B = 1.2

#attenuation true numbers
alpha = 0.919
beta = -1.13

#number of events to generate
events = 10000
#number of bins for the zenith 
nr_of_bins = 8

data = generate_toy_data(events, minE, maxE, gamma, A, B, alpha, beta)

#print(data)

In [ ]:
fig1, ax1 =plt.subplots(figsize=(10,6))
plt.grid()

ax1.hist(data.cos2,bins=100 )


In [ ]:
#### fit the attenuation curve
data = set_intensity(data, nr_of_bins)
#print(data)        
    
### input value the reference intesity
intensity = 100
s125_fit, bins2, s38_fit , s125_fit_error= get_data_to_fit(data, intensity, nr_of_bins)
print(bins2)
values, cov = get_attenuation_parameters(s125_fit, bins2, performMCMC = False )

print(values)

In [ ]:
fig2, ax2 =plt.subplots(figsize=(10,6))
plt.grid()
from simul_functions import get_s125 
a1=values[0]
a2=values[1]
a3=values[2]
y=get_s125(bins2, a1,a2,a3)
y2=get_s125(bins2, alpha, beta, a3)

ax2.plot(bins2, s125_fit, lw=0, marker='o')
ax2.plot(bins2,y, color="r")
ax2.plot(bins2,y2)

In [ ]:
#groups = dict(list(data))
#print(groups)


    
fig1, ax1 =plt.subplots(figsize=(10,6))
plt.grid()

ax1.plot(np.log10(data.s125),data.I, lw=0, marker='o' )
ax1.set_yscale('log')
ax1.set_xlim(2,4)



In [ ]:
### number of boostrap samples as input
samples = 2
from simul_functions import get_s125

fig2, ax2 = plt.subplots(figsize=(10,6))
plt.grid()
a=[alpha,beta,1]
bootstrap_values=[]
bootstrap_values_2=[]
for j in range(0, samples):
    new_data = get_bootstrap_data(data)
    new_data=set_intensity(new_data, nr_of_bins)
    s125_fit, bins, s38_fit, s125_fit_error = get_data_to_fit(new_data, intensity, nr_of_bins)
    vals, cov_bt = get_attenuation_parameters(s125_fit, bins, performMCMC = False )
    bootstrap_values.append(vals)
    bootstrap_values_2.append(vals**2)
    ax2.plot(bins, s125_fit, lw=0, marker='o')
    ax2.plot(bins, get_s125(bins,vals[0], vals[1], vals[2]))
    print(vals)


Things to do:
1. From the bootstrap get all the values and plot them in an histogram and one vs each other (in one canvas)
2. Give final results: values(from original data) +- rms(vals)
3. Implement MCMC fit in the get_attenuation_parameters function
4. Compare results with 2.
5. See/write to Ioana/Zig

In [ ]:
from simul_functions import bootstrap_graphs

plot=bootstrap_graphs(bootstrap_values, alpha, beta)

In [ ]:
plot.savefig("bootstrap_values2")

In [ ]:
#final results with error
from simul_functions import rms_for_initial_vals

error=rms_for_initial_vals(bootstrap_values, bootstrap_values_2)
final_values_up=values + error
final_values_down=values - error

print(values,'+/-',error)
print(final_values_up)

In [ ]:
from simul_functions import get_attenuation_parameters2

final_parameters, cov2, sample, sample2 = get_attenuation_parameters2(s125_fit, s38_fit,s125_fit_error, bins, performMCMC=True)

In [ ]:
#print(final_parameters)
#print(sample)
print(sample)
print(sample2)

In [ ]:
print(a3)
print(sample2)
import corner
fig = corner.corner(sample2, labels=["$a$","$b$", "$s38$"], truths=[alpha,beta, a3])
fig.savefig("")
print(sample2)

In [ ]:
a_mcmc, b_mcmc, s38_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(sample2, [16, 50, 84],
                                                axis=0)))
print(a_mcmc, b_mcmc, s38_mcmc)
#(mid_value, +error, -error)
print("a = %f + %f - %f\n"%(a_mcmc[0],a_mcmc[1],a_mcmc[2]))
print("b = %f + %f - %f\n"%(b_mcmc[0],b_mcmc[1],b_mcmc[2]))
print("s38 = %f + %f - %f\n"%(s38_mcmc[0],s38_mcmc[1],s38_mcmc[2]))